In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl
from matplotlib.ticker import AutoLocator, AutoMinorLocator, LogLocator
from matplotlib.colors import Normalize
import glob
from scipy.interpolate import griddata
from pathlib import Path
import h5py
import sys
from matplotlib.lines import Line2D
from matplotlib.legend_handler import HandlerTuple
from matplotlib.ticker import MultipleLocator

# Where am I running?
try:
    # Normal script
    here = Path(__file__).resolve().parent
except NameError:
    # Notebook / REPL
    here = Path.cwd()

phys_const_path = (here / '..' / 'phys_const').resolve()
sys.path.append(str(phys_const_path))

nsm_plots_path = (here / '..' / 'nsm_plots').resolve()
sys.path.append(str(nsm_plots_path))

nsm_plots_postproc = (here / '..' / 'nsm_instabilities').resolve()
sys.path.append(str(nsm_plots_postproc))

import phys_const as pc
import plot_functions as pf
import functions_angular_crossings as fac

### Reading hdf5 data

In [ ]:
with h5py.File(f'/home/erick/software/devscrpts/gw170817_paper_plots/1506_ppEb_0.32ms_sigmaFFI_slices.h5', 'r') as f:

    xz_x_km_ffi = f['xz_x_km'][:]
    xz_y_km_ffi = f['xz_y_km'][:]
    xz_z_invsec_ffi = f['xz_z_invsec'][:]

    xy_x_km_ffi = f['xy_x_km'][:]
    xy_y_km_ffi = f['xy_y_km'][:]
    xy_z_invsec_ffi = f['xy_z_invsec'][:]


with h5py.File(f'/home/erick/software/devscrpts/gw170817_paper_plots/1506_ppEb_0.32ms_sigmaCFI_slices.h5', 'r') as f:

    xz_x_km_cfi = f['xz_x_km'][:]
    xz_y_km_cfi = f['xz_y_km'][:]
    xz_z_invsec_cfi = f['xz_z_invsec'][:]

    xy_x_km_cfi = f['xy_x_km'][:]
    xy_y_km_cfi = f['xy_y_km'][:]
    xy_z_invsec_cfi = f['xy_z_invsec'][:]

In [ ]:
bh_r=5.43, # km
bh_x=48.0, # km
bh_y=48.0, # km
bh_z=16.0, # km

In [ ]:
cmap = 'plasma'

# profile slices
temp_slice_up  = np.array(xy_z_invsec_ffi)
ye_slice_up    = np.array(xy_z_invsec_cfi)
temp_slice_low = np.array(xz_z_invsec_ffi)
ye_slice_low   = np.array(xz_z_invsec_cfi)

xup = np.array(xy_x_km_ffi)
yup = np.array(xy_y_km_ffi)
xlow = np.array(xz_x_km_ffi)
ylow = np.array(xz_y_km_ffi)

# color bar limits
temp_max = np.max([np.nanmax(temp_slice_low), np.nanmax(temp_slice_up)])
temp_min = np.min([np.nanmin(temp_slice_low), np.nanmin(temp_slice_up)])
ye_max = np.max([np.nanmax(ye_slice_low), np.nanmax(ye_slice_up)])
ye_min = np.min([np.nanmin(ye_slice_low), np.nanmin(ye_slice_up)])
xmid = 96 / 2 # km
ymid = 96 / 2
zmid = 32 / 2 # km

# create 2x2 figure
fig, axes = plt.subplots(2, 2, figsize=(12, 12), sharex='col', sharey='row')
plt.subplots_adjust(wspace=0, hspace=-0.19)
fig.align_labels()

import mpl_toolkits

# colorbars
norm = mpl.colors.Normalize(vmin=temp_min, vmax=temp_max)
sm = mpl.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])  # Required for ScalarMappable even if unused
divider = mpl_toolkits.axes_grid1.make_axes_locatable(axes[0,0])
cax = divider.append_axes("top", size="8%", pad=0.15)  # <-- Increase pad value to move colorbar up
cbar = plt.colorbar(sm, cax=cax, orientation='horizontal')
cbar.ax.tick_params(which="both",direction="in")
cbar.ax.xaxis.set_ticks_position('top')
cbar.ax.xaxis.set_label_position('top')
cbar.ax.xaxis.set_minor_locator(AutoMinorLocator())
cbar.set_label(r"$\log \sigma_{\mathrm{FFI}} \,[1/\mathrm{s}]$", labelpad=10)
cax.xaxis.set_major_locator(MultipleLocator(1))

norm = mpl.colors.Normalize(vmin=ye_min, vmax=ye_max)
sm = mpl.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])  # Required for ScalarMappable even if unused
divider = mpl_toolkits.axes_grid1.make_axes_locatable(axes[0,1])
cax = divider.append_axes("top", size="8%", pad=0.15)  # <-- Increase pad value here too
cbar = plt.colorbar(sm, cax=cax, orientation='horizontal')
cbar.ax.tick_params(which="both",direction="in")
cbar.ax.xaxis.set_ticks_position('top')
cbar.ax.xaxis.set_label_position('top')
cbar.ax.xaxis.set_minor_locator(AutoMinorLocator())
cbar.set_label(r"$\log \sigma_{\mathrm{CFI}} \,[1/\mathrm{s}]$", labelpad=10)

# set axis labels
for ax in axes[-1,:]:
    ax.set_xlabel(r'$x\,[\mathrm{km}]$')
axes[0,0].set_ylabel(r'$z\,[\mathrm{km}]$')
axes[1,0].set_ylabel(r'$y\,[\mathrm{km}]$')
# remove xtick labels on all but the bottom row
for ax in axes[:-1,:].flatten():
    for label in ax.get_xticklabels():
        label.set_visible(False) 
for ax in axes[:,1:].flatten():
    for label in ax.get_yticklabels():
        label.set_visible(False)

for ax in axes.flat:
    ax.set_aspect('equal')
    ax.xaxis.set_major_locator(MultipleLocator(20))
    ax.yaxis.set_major_locator(MultipleLocator(20))
    # ax.tick_params(axis='both', which='both', direction='in', top=True, right=True)
    ax.minorticks_on()
    # pf.apply_custom_settings(ax, False)

# mask out the black hole region
axes[0,0].pcolormesh(xlow-xmid, ylow-zmid, temp_slice_low, shading='auto', cmap=cmap, vmin=temp_min, vmax=temp_max)
circle_bh = plt.Circle((bh_x[0]-xmid, bh_z[0]-zmid), bh_r[0], color='k', fill=False, linestyle='dashed', linewidth=1)
axes[0,0].add_patch(circle_bh)

axes[1,0].pcolormesh(xup-xmid , yup-ymid , temp_slice_up , shading='auto', cmap=cmap, vmin=temp_min, vmax=temp_max)
circle_bh = plt.Circle((bh_x[0]-xmid, bh_y[0]-ymid), bh_r[0], color='k', fill=False, linestyle='dashed', linewidth=1)
axes[1,0].add_patch(circle_bh)
# Add legend for black hole horizon
legend_circle = Line2D([0], [0], color='k', linestyle='dashed', linewidth=1, label='Black hole')
axes[1,0].legend(handles=[legend_circle], loc='upper right', frameon=False, fontsize=14)

axes[0,1].pcolormesh(xlow-xmid, ylow-zmid, ye_slice_low  , shading='auto', cmap=cmap, vmin=ye_min  , vmax=ye_max  )
circle_bh = plt.Circle((bh_x[0]-xmid, bh_z[0]-zmid), bh_r[0], color='k', fill=False, linestyle='dashed', linewidth=1)
axes[0,1].add_patch(circle_bh)

axes[1,1].pcolormesh(xup-xmid , yup-ymid , ye_slice_up   , shading='auto', cmap=cmap, vmin=ye_min  , vmax=ye_max  )
circle_bh = plt.Circle((bh_x[0]-xmid, bh_y[0]-ymid), bh_r[0], color='k', fill=False, linestyle='dashed', linewidth=1)
axes[1,1].add_patch(circle_bh)

plt.savefig('plots/global_instabilities.pdf', bbox_inches='tight')

